In [1]:
!pip install -qU \
  langchain-core==0.3.33 \
  langchain-groq \
  langchain-community==0.3.16

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.7/412.7 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 73.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langgraph-prebuilt 1.0.7 requires langchain-core>=1.0.0, but you have langchain-core 0.3.33 which is incompatible.


In [2]:
import os
from getpass import getpass
from langchain_groq import ChatGroq

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY") or \
    getpass("Enter your GROQ API key: ")

Enter your GROQ API key: ··········


In [3]:
llm = ChatGroq(temperature=0.0, model="llama-3.3-70b-versatile", streaming=True)

In [4]:
llm_out = llm.invoke("Hello there")
llm_out

AIMessage(content="Hello. It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run-f2fa0006-8c65-4aad-a620-86a7a65af39e-0', usage_metadata={'input_tokens': 37, 'output_tokens': 25, 'total_tokens': 62})

# Streaming with astream


In [5]:
tokens = []
async for token in llm.astream("What is NLP?"):
  tokens.append(token)
  print(token.content, end="!", flush=True)

!N!LP! stands! for! Natural! Language! Processing!,! which! is! a! sub!field! of! artificial! intelligence! (!AI!)! that! deals! with! the! interaction! between! computers! and! humans! in! natural! language!.! It! is! a! multid!isc!iplinary! field! that! combines! computer! science!,! lingu!istics!,! and! cognitive! psychology! to! enable! computers! to! process!,! understand!,! and! generate! human! language!.

!N!LP! involves! a! range! of! techniques! and! technologies!,! including!:

!1!.! **!Text! analysis!**:! extracting! meaning! and! insights! from! text! data!,! such! as! sentiment! analysis!,! entity! recognition!,! and! topic! modeling!.
!2!.! **!Language! understanding!**:! enabling! computers! to! comprehend! the! meaning! of! text!,! including! syntax!,! semantics!,! and! prag!m!atics!.
!3!.! **!Language! generation!**:! generating! human!-like! text!,! such! as! chat!bots!,! language! translation!,! and! text! summar!ization!.
!4!.! **!Speech! recognition!**:! convertin

In [6]:
tokens[0]

AIMessageChunk(content='', additional_kwargs={}, response_metadata={}, id='run-20bc6d37-ea1b-4cc9-8dbb-6f389fa7bc07')

In [7]:
tokens[1]

AIMessageChunk(content='N', additional_kwargs={}, response_metadata={}, id='run-20bc6d37-ea1b-4cc9-8dbb-6f389fa7bc07')

In [8]:
tokens[0] + tokens[1] + tokens[2] + tokens[3] + tokens[4]

AIMessageChunk(content='NLP stands for', additional_kwargs={}, response_metadata={}, id='run-20bc6d37-ea1b-4cc9-8dbb-6f389fa7bc07')

# Streaming with Agents

To construct the agent executor we need:

1. Tools
2. ChatPromptTemplate
3. Our LLM (already defined with llm)
4. An agent
5. Finally, the agent executor

## Tools

In [9]:
from langchain_core.tools import tool

@tool
def add(x: float, y: float) -> float:
    """Add 'x' and 'y'."""
    return x + y

@tool
def multiply(x: float, y: float) -> float:
    """Multiply 'x' and 'y'."""
    return x * y

@tool
def exponentiate(x: float, y: float) -> float:
    """Raise 'x' to the power of 'y'."""
    return x ** y

@tool
def subtract(x: float, y: float) -> float:
    """Subtract 'x' from 'y'."""
    return y - x

@tool
def final_answer(answer: str, tools_used: list[str]) -> str:
    """Use this tool to provide a final answer to the user.
    The answer should be in natural language as this will be provided
    to the user directly. The tools_used must include a list of tool
    names that were used within the `scratchpad`. You MUST use this tool
    to conclude the interaction.
    """
    return {"answer": answer, "tools_used": tools_used}

In [10]:
tools = [add, multiply, exponentiate, subtract, final_answer]

## ChatPromptTemplate

In [11]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "You're a helpful assistant. When answering a user's question "
        "you should first use one of the tools provided. After using a "
        "tool the tool output will be provided back to you. You MUST "
        "then use the final_answer tool to provide a final answer to the user. "
        "DO NOT use the same tool more than once."
    )),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

## Agent

In [12]:
from langchain_core.runnables.base import RunnableSerializable

tools = [add, subtract, multiply, exponentiate, final_answer]

# define the agent runnable
agent: RunnableSerializable = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: x["chat_history"],
        "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
    }
    | prompt
    | llm.bind_tools(tools, tool_choice="any")
)

## Agent Executor

In [13]:
import json
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage


# create tool name to function mapping
name2tool = {tool.name: tool.func for tool in tools}

class CustomAgentExecutor:
    chat_history: list[BaseMessage]

    def __init__(self, max_iterations: int = 3):
        self.chat_history = []
        self.max_iterations = max_iterations
        self.agent: RunnableSerializable = (
            {
                "input": lambda x: x["input"],
                "chat_history": lambda x: x["chat_history"],
                "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
            }
            | prompt
            | llm.bind_tools(tools, tool_choice="any")  # we're forcing tool use again
        )

    def invoke(self, input: str) -> dict:
        # invoke the agent but we do this iteratively in a loop until
        # reaching a final answer
        count = 0
        agent_scratchpad = []
        while count < self.max_iterations:
            # invoke a step for the agent to generate a tool call
            out = self.agent.invoke({
                "input": input,
                "chat_history": self.chat_history,
                "agent_scratchpad": agent_scratchpad
            })
            # if the tool call is the final answer tool, we stop
            if out.tool_calls[0]["name"] == "final_answer":
                break
            agent_scratchpad.append(out)  # add tool call to scratchpad
            # otherwise we execute the tool and add it's output to the agent scratchpad
            tool_out = name2tool[out.tool_calls[0]["name"]](**out.tool_calls[0]["args"])
            # add the tool output to the agent scratchpad
            action_str = f"The {out.tool_calls[0]['name']} tool returned {tool_out}"
            agent_scratchpad.append({
                "role": "tool",
                "content": action_str,
                "tool_call_id": out.tool_calls[0]["id"]
            })
            # add a print so we can see intermediate steps
            print(f"{count}: {action_str}")
            count += 1
        # add the final output to the chat history
        final_answer = out.tool_calls[0]["args"]
        # this is a dictionary, so we convert it to a string for compatibility with
        # the chat history
        final_answer_str = json.dumps(final_answer)
        self.chat_history.append({"input": input, "output": final_answer_str})
        self.chat_history.extend([
            HumanMessage(content=input),
            AIMessage(content=final_answer_str)
        ])
        # return the final answer in dict form
        return final_answer

agent_executor = CustomAgentExecutor()

In [14]:
agent_executor.invoke(input="What is 10 + 10")

0: The add tool returned 20


{'answer': 'The answer to 10 + 10 is 20.', 'tools_used': ['add']}

Let's modify our agent_executor to use streaming and parse the streamed output into a format that we can more easily work with.

In [15]:
from langchain_core.runnables import ConfigurableField

llm = ChatGroq(
    model_name="llama-3.3-70b-versatile",
    temperature=0.0,
    streaming=True
).configurable_fields(
    callbacks=ConfigurableField(
        id="callbacks",
        name="callbacks",
        description="A list of callbacks to use for streaming",
    )
)

We reinitialize our agent, nothing changes here:

In [16]:
agent: RunnableSerializable = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: x["chat_history"],
        "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
    }
    | prompt
    | llm.bind_tools(tools, tool_choice="any")
)

In [17]:
import asyncio
from langchain.callbacks.base import AsyncCallbackHandler


class QueueCallbackHandler(AsyncCallbackHandler):
    """Callback handler that puts tokens into a queue."""

    def __init__(self, queue: asyncio.Queue):
        self.queue = queue
        self.final_answer_seen = False

    async def __aiter__(self):
        while True:
            if self.queue.empty():
                await asyncio.sleep(0.1)
                continue
            token_or_done = await self.queue.get()

            if token_or_done == "<<DONE>>":
                # this means we're done
                return
            if token_or_done:
                yield token_or_done

    async def on_llm_new_token(self, *args, **kwargs) -> None:
        """Put new token in the queue."""
        #print(f"on_llm_new_token: {args}, {kwargs}")
        chunk = kwargs.get("chunk")
        if chunk:
            # check for final_answer tool call
            if tool_calls := chunk.message.additional_kwargs.get("tool_calls"):
                if tool_calls[0]["function"]["name"] == "final_answer":
                    # this will allow the stream to end on the next `on_llm_end` call
                    self.final_answer_seen = True
        await self.queue.put(chunk)
        return

    async def on_llm_end(self, *args, **kwargs) -> None:
        """Put None in the queue to signal completion."""
        #print(f"on_llm_end: {args}, {kwargs}")
        # this should only be used at the end of our agent execution, however LangChain
        # will call this at the end of every tool call, not just the final tool call
        # so we must only send the "done" signal if we have already seen the final_answer
        # tool call
        if self.final_answer_seen:
            await self.queue.put("<<DONE>>")
        else:
            await self.queue.put("<<STEP_END>>")
        return

In [18]:
queue = asyncio.Queue()
streamer = QueueCallbackHandler(queue)

tokens = []

async def stream(query: str):
    response = agent.with_config(
        callbacks=[streamer]
    )
    async for token in response.astream({
        "input": query,
        "chat_history": [],
        "agent_scratchpad": []
    }):
        tokens.append(token)
        print(token, flush=True)

await stream("What is 10 + 10")

content='' additional_kwargs={} response_metadata={} id='run-460425d2-e510-4b1c-a006-66f96aa41e5a'
content='' additional_kwargs={'tool_calls': [{'index': 0, 'id': '4ech362jv', 'function': {'arguments': '{"x":10,"y":10}', 'name': 'add'}, 'type': 'function'}]} response_metadata={} id='run-460425d2-e510-4b1c-a006-66f96aa41e5a' tool_calls=[{'name': 'add', 'args': {'x': 10, 'y': 10}, 'id': '4ech362jv', 'type': 'tool_call'}] tool_call_chunks=[{'name': 'add', 'args': '{"x":10,"y":10}', 'id': '4ech362jv', 'index': 0, 'type': 'tool_call_chunk'}]
content='' additional_kwargs={} response_metadata={'finish_reason': 'tool_calls'} id='run-460425d2-e510-4b1c-a006-66f96aa41e5a' usage_metadata={'input_tokens': 687, 'output_tokens': 16, 'total_tokens': 703}


In [19]:
tk = tokens[0]

for token in tokens[1:]:
    tk += token

tk

AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': '4ech362jv', 'function': {'arguments': '{"x":10,"y":10}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'}, id='run-460425d2-e510-4b1c-a006-66f96aa41e5a', tool_calls=[{'name': 'add', 'args': {'x': 10, 'y': 10}, 'id': '4ech362jv', 'type': 'tool_call'}], usage_metadata={'input_tokens': 687, 'output_tokens': 16, 'total_tokens': 703}, tool_call_chunks=[{'name': 'add', 'args': '{"x":10,"y":10}', 'id': '4ech362jv', 'index': 0, 'type': 'tool_call_chunk'}])

In [20]:
from langchain_core.messages import ToolMessage

class CustomAgentExecutor:
    chat_history: list[BaseMessage]

    def __init__(self, max_iterations: int = 3):
        self.chat_history = []
        self.max_iterations = max_iterations
        self.agent: RunnableSerializable = (
            {
                "input": lambda x: x["input"],
                "chat_history": lambda x: x["chat_history"],
                "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
            }
            | prompt
            | llm.bind_tools(tools, tool_choice="any")  # we're forcing tool use again
        )

    async def invoke(self, input: str, streamer: QueueCallbackHandler, verbose: bool = False) -> dict:
        # invoke the agent but we do this iteratively in a loop until
        # reaching a final answer
        count = 0
        agent_scratchpad = []
        while count < self.max_iterations:
            # invoke a step for the agent to generate a tool call
            async def stream(query: str):
                response = self.agent.with_config(
                    callbacks=[streamer]
                )
                # we initialize the output dictionary that we will be populating with
                # our streamed output
                output = None
                # now we begin streaming
                async for token in response.astream({
                    "input": query,
                    "chat_history": self.chat_history,
                    "agent_scratchpad": agent_scratchpad
                }):
                    if output is None:
                        output = token
                    else:
                        # we can just add the tokens together as they are streamed and
                        # we'll have the full response object at the end
                        output += token
                    if token.content != "":
                        # we can capture various parts of the response object
                        if verbose: print(f"content: {token.content}", flush=True)
                    tool_calls = token.additional_kwargs.get("tool_calls")
                    if tool_calls:
                        if verbose: print(f"tool_calls: {tool_calls}", flush=True)
                        tool_name = tool_calls[0]["function"]["name"]
                        if tool_name:
                            if verbose: print(f"tool_name: {tool_name}", flush=True)
                        arg = tool_calls[0]["function"]["arguments"]
                        if arg != "":
                            if verbose: print(f"arg: {arg}", flush=True)
                return AIMessage(
                    content=output.content,
                    tool_calls=output.tool_calls,
                    tool_call_id=output.tool_calls[0]["id"]
                )

            tool_call = await stream(query=input)
            # add initial tool call to scratchpad
            agent_scratchpad.append(tool_call)
            # otherwise we execute the tool and add it's output to the agent scratchpad
            tool_name = tool_call.tool_calls[0]["name"]
            tool_args = tool_call.tool_calls[0]["args"]
            tool_call_id = tool_call.tool_call_id
            tool_out = name2tool[tool_name](**tool_args)
            # add the tool output to the agent scratchpad
            tool_exec = ToolMessage(
                content=f"{tool_out}",
                tool_call_id=tool_call_id
            )
            agent_scratchpad.append(tool_exec)
            count += 1
            # if the tool call is the final answer tool, we stop
            if tool_name == "final_answer":
                break
        # add the final output to the chat history, we only add the "answer" field
        final_answer = tool_out["answer"]
        self.chat_history.extend([
            HumanMessage(content=input),
            AIMessage(content=final_answer)
        ])
        # return the final answer in dict form
        return tool_args

agent_executor = CustomAgentExecutor()

In [21]:
queue = asyncio.Queue()
streamer = QueueCallbackHandler(queue)

out = await agent_executor.invoke("What is 10 + 10", streamer, verbose=True)

tool_calls: [{'index': 0, 'id': 'tbe6bv465', 'function': {'arguments': '{"x":10,"y":10}', 'name': 'add'}, 'type': 'function'}]
tool_name: add
arg: {"x":10,"y":10}
tool_calls: [{'index': 0, 'id': 'paj6wnamg', 'function': {'arguments': '{"answer":"The answer to 10 + 10 is 20.","tools_used":["add"]}', 'name': 'final_answer'}, 'type': 'function'}]
tool_name: final_answer
arg: {"answer":"The answer to 10 + 10 is 20.","tools_used":["add"]}


In [22]:
queue = asyncio.Queue()
streamer = QueueCallbackHandler(queue)

out = await agent_executor.invoke("What is 10 + 10", streamer)

In [23]:
queue = asyncio.Queue()
streamer = QueueCallbackHandler(queue)

task = asyncio.create_task(agent_executor.invoke("What is 10 + 10", streamer))

async for token in streamer:
    print(token, flush=True)

await task

message=AIMessageChunk(content='', additional_kwargs={}, response_metadata={}, id='run-e1f4dabf-bf76-4b15-95f9-8b32ad18e724')
message=AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': '95c07t9np', 'function': {'arguments': '{"x":10,"y":10}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={}, id='run-e1f4dabf-bf76-4b15-95f9-8b32ad18e724', tool_calls=[{'name': 'add', 'args': {'x': 10, 'y': 10}, 'id': '95c07t9np', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'add', 'args': '{"x":10,"y":10}', 'id': '95c07t9np', 'index': 0, 'type': 'tool_call_chunk'}])
generation_info={'finish_reason': 'tool_calls'} message=AIMessageChunk(content='', additional_kwargs={}, response_metadata={'finish_reason': 'tool_calls'}, id='run-e1f4dabf-bf76-4b15-95f9-8b32ad18e724', usage_metadata={'input_tokens': 744, 'output_tokens': 16, 'total_tokens': 760})
<<STEP_END>>
message=AIMessageChunk(content='', additional_kwargs={}, response_metadata={}, id='run-50c6cce3-3c4d-4a85

{'answer': 'The answer to 10 + 10 is 20', 'tools_used': ['add']}

In [24]:
queue = asyncio.Queue()
streamer = QueueCallbackHandler(queue)

task = asyncio.create_task(agent_executor.invoke("What is 10 + 10", streamer))

async for token in streamer:
    # first identify if we have a <<STEP_END>> token
    if token == "<<STEP_END>>":
        print("\n", flush=True)
    # we'll first identify if the token is a tool call
    elif tool_calls := token.message.additional_kwargs.get("tool_calls"):
        # if we have a tool call with a tool name, we'll print it
        if tool_name := tool_calls[0]["function"]["name"]:
            print(f"Calling {tool_name}...", flush=True)
        # if we have a tool call with arguments, we ad them to our args string
        if tool_args := tool_calls[0]["function"]["arguments"]:
            print(f"{tool_args}", end="", flush=True)

_ = await task

Calling add...
{"x":10,"y":10}

Calling final_answer...
{"answer":"The answer to 10 + 10 is 20","tools_used":["add"]}